In [1]:
# 1. Cài đặt thư viện
print("⏳ Bắt đầu cài đặt thư viện...")
%pip install ultralytics gradio -q
print("✅ Cài đặt hoàn tất!")

import gradio as gr
from ultralytics import YOLO
from PIL import Image
import os

# --- Custom CSS để giao diện đẹp hơn ---
custom_css = """
@import url('https://fonts.googleapis.com/css2?family=Quicksand:wght@400;500;700&display=swap');
body, .gradio-container { font-family: 'Quicksand', sans-serif; }
#component-0 { background: linear-gradient(135deg, #e0c3fc 0%, #8ec5fc 100%); }
.panel {
    box-shadow: 0 4px 12px 0 rgba(0,0,0,0.1);
    border-radius: 15px !important;
    transition: box-shadow 0.3s ease-in-out;
}
.panel:hover { box-shadow: 0 8px 24px 0 rgba(0,0,0,0.15); }
#disclaimer { color: #555; }
"""

# --- CẤU HÌNH ---
model_path = '/content/best (1).pt'
age_group_info = {
    0: {
        "name": "Teenagers", "range": "(15-30 tuổi)", "emoji": "🧑",
        "description": "Bàn tay này cho thấy một tâm hồn trẻ trung, đầy năng lượng và luôn sẵn sàng khám phá những điều mới mẻ."
    },
    1: {
        "name": "Adults", "range": "(30-55 tuổi)", "emoji": "👩‍🦰",
        "description": "Đây là bàn tay của sự trưởng thành, thể hiện kinh nghiệm và sự vững chãi, là trụ cột vững chắc cho gia đình và xã hội."
    },
    2: {
        "name": "Seniors", "range": "(55-80+ tuổi)", "emoji": "🧓",
        "description": "Bàn tay này chứa đựng sự thông thái và bình yên. Họ đã tích lũy được vốn sống quý báu và giờ đây tận hưởng sự thanh thản."
    }
}

# --- Tải mô hình ---
print(f"⏳ Đang tải mô hình từ file '{model_path}'...")
model = YOLO(model_path)
print("✅ Tải mô hình thành công!")

# --- Hàm xử lý ---
def analyze_hand(input_image):
    if input_image is None:
        return None, None, "<p style='text-align: center; color: #999;'>Hãy tải ảnh lên và bắt đầu phân tích!</p>"

    results = model(Image.fromarray(input_image), conf=0.45)
    result = results[0]

    if len(result.boxes) == 0:
        return input_image, None, "<h3 style='color: #E74C3C; text-align: center;'>PalmSense không nhận diện được bàn tay nào. <br>Vui lòng thử ảnh khác rõ nét hơn.</h3>"

    annotated_image = result.plot()[..., ::-1]
    box = result.boxes[0]
    class_id = int(box.cls)
    confidence = float(box.conf)
    info = age_group_info.get(class_id, {"name": "Không xác định", "range": "", "emoji": "❓", "description": ""})

    confidence_percent = confidence * 100
    confidence_html = f"""
    <div style='background-color: #e9ecef; border-radius: 5px; padding: 3px; margin-top: 10px;'>
        <div style='width: {confidence_percent}%; background: linear-gradient(90deg, #1cb5e0 0%, #000046 100%); color: white; text-align: center; border-radius: 5px; padding: 5px; font-weight: bold;'>
            {confidence:.2%}
        </div>
    </div>
    """

    final_text = f"""
    <div style='padding: 20px;'>
        <h2 style='color: #2C3E50; text-align: center;'>{info['emoji']} PalmSense Dự Đoán {info['emoji']}</h2>
        <hr>
        <p style='font-size: 1.2em;'><strong>Nhóm tuổi:</strong> <span style='color: #2980B9; font-weight: bold;'>{info['name']} {info['range']}</span></p>
        <p style='font-style: italic; color: #566573;'>"{info['description']}"</p>
        <p style='font-size: 1.1em; margin-top: 15px;'><strong>Độ tin cậy của mô hình:</strong></p>
        {confidence_html}
    </div>
    """
    return annotated_image, annotated_image, final_text

def clear_all():
    return None, None, "<p style='text-align: center; color: #999;'>Sẵn sàng cho lần phân tích tiếp theo!</p>"

# --- Thiết kế giao diện ---
with gr.Blocks(theme=gr.themes.Soft(primary_hue="blue", secondary_hue="sky"), css=custom_css, title="PalmSense 2.0") as demo:
    gr.Markdown("""
    <div style="text-align: center;">
        <h1 style="color: #2C3E50;">🖐️ Welcome to PalmSense 2.0 🖐️</h1>
        <p style="color: #566573; font-size: 1.2em;">Nâng cấp trải nghiệm phân tích bàn tay với AI</p>
    </div>
    """)

    with gr.Row(variant='panel'):
        with gr.Column(scale=1):
            image_input = gr.Image(type="numpy", label="1. Tải ảnh của bạn lên đây", height=400)
            with gr.Row():
                btn_clear = gr.Button("✨ Bắt đầu lại")
                btn_analyze = gr.Button("🔮 Phân Tích Ngay!", variant="primary")

        with gr.Column(scale=1):
            image_output = gr.Image(label="Kết quả nhận diện", interactive=False, height=400)
            text_output = gr.HTML(label="Thông tin chi tiết")

    gr.Examples(
        examples=[["/content/image_001.jpg"], ["/content/image_075.jpg"],["/content/image_120.jpg"]],
        inputs=image_input, label="Hoặc thử với các ví dụ sau"
    )

    gr.Markdown(
        """<p id="disclaimer" style="text-align: center; font-size: 0.8em; margin-top: 20px;">
        <strong>Lưu ý:</strong> Kết quả chỉ mang tính chất tham khảo và giải trí.
        </p>""",
    )

    btn_analyze.click(fn=analyze_hand, inputs=image_input, outputs=[image_output, image_output, text_output])
    btn_clear.click(fn=clear_all, inputs=None, outputs=[image_input, image_output, text_output])

# --- Khởi chạy ---
demo.launch(debug=True, share=True)

⏳ Bắt đầu cài đặt thư viện...
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 17.9 MB/s eta 0:00:00
✅ Cài đặt hoàn tất!
Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
⏳ Đang tải mô hình từ file '/content/best (1).pt'...
✅ Tải mô hình thành công!
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://504a9859e61a0f0c7c.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)



0: 640x640 1 Seniors (55-80+), 370.1ms
Speed: 31.0ms preprocess, 370.1ms inference, 32.1ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 Teenagers ( 15-30), 1 Seniors (55-80+), 179.8ms
Speed: 2.9ms preprocess, 179.8ms inference, 1.8ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 (no detections), 132.3ms
Speed: 9.1ms preprocess, 132.3ms inference, 0.5ms postprocess per image at shape (1, 3, 384, 640)

0: 640x640 1 Adults ( 30-55), 278.7ms
Speed: 5.6ms preprocess, 278.7ms inference, 1.3ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 Seniors (55-80+), 177.7ms
Speed: 2.6ms preprocess, 177.7ms inference, 1.1ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 Teenagers ( 15-30), 1 Seniors (55-80+), 177.0ms
Speed: 2.9ms preprocess, 177.0ms inference, 1.2ms postprocess per image at shape (1, 3, 640, 640)
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://504a9859e61a0f0c7c.gradio.live
